<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Ada Tech [DS-PY-004] Técnicas de Programação I (PY) Aulas 4 e 5 : Pandas - Enunciado do Exercício 3.

## Intro

Nesta parte vimos alguns métodos do Pandas para calcular medidas por grupos.

Para os exercícios, usaremos um conjunto de dados de propriedades à venda na Cidade de Buenos Aires em 2016 disponibilizado pelo GCBA.

A ideia dessa prática é revisar alguns tópicos que vimos nas aulas anteriores e exercitar os diferentes métodos que vimos nesta prática e que vamos usar novamente em aulas futuras.

## Dataset

O Governo da Cidade de Buenos Aires disponibiliza alguns [conjuntos de dados](https://data.buenosaires.gob.ar/) para acesso público

Lá encontramos dados sobre [imóveis à venda](https://data.buenosaires.gob.ar/dataset/departamentos-venta), separados por ano.

Nesta aula, vamos usar um subconjunto dos dados de [Apartamentos à venda 2016](https://data.buenosaires.gob.ar/dataset/departamentos-venta/archivo/juqdkmgo-7031-resource) que baixamos.

Na pasta Dados desta aulas, já temos esse conjunto de dados baixado, o nome do arquivo é **departamentos-em-venda-2016.csv**.

## Imports

In [4]:
import pandas as pd
import numpy as np

## Exercício 1  - Importar 

Vamos ler os dados do arquivo `departamentos-em-venda-2016.csv`.

Vamos ver quantos registros você possui e quais são os tipos de dados em cada coluna.

Vejamos os primeiros registros para verificar se os dados foram importados corretamente.

In [5]:
df = pd.read_csv("../data/departamentos-em-venda-2016.csv", sep=";")

## Exercício 2  - Limpeza

Vamos criar uma nova coluna (PRECIOTEXT_CLEAN) do tipo numérico, que tem os valores em dólares que lemos no campo PRECIOTEXT, ou NaN se esses valores forem expressos em pesos.

1) Crie um objeto Series que resulte da limpeza dos valores do campo PRECIOTEXT. Temos que remover os símbolos "U\\$S" e "."

2) No caso da Series resultante do ponto 1) substituímos os valores em pesos ('\\$') por nulos.

3) Crie uma nova coluna de tipo numérico no DataFrame (PRECIOTEXT_CLEAN) e atribua os valores de resultado de 2)

In [6]:
df = df.assign(PRECIOTEXT_CLEAN=df["PRECIOTEXT"].str.replace(".", "").str.extract(r"(\d+)").astype(float))

## Exercício 3  - Preço por metro quadrado em dólares

Vamos criar uma coluna do tipo numérico (PRECIOTEXTM) que tem o preço do $m^{2}$ em dólares calculado como o valor de PRECIOTEXT_CLEAN / M2.

In [7]:
df = df.assign(PRECIOTEXTM=lambda d: d["PRECIOTEXT_CLEAN"] / d["M2"])

## Exercício 4  - Preço médio por metro quadrado em dólares por bairro

Vamos calcular o preço médio por metro quadrado em dólares por bairro usando `groupby` e `pivot_table`.

Quais são os cinco bairros mais caros?

In [8]:
df.columns

Index(['CALLE', 'NUMERO', 'ID_ZONAPRO', 'OPERACION', 'TIPO', 'M2', 'M2CUB',
       'PRECIOTEXT', 'PRECIOARS', 'PRECIOARSM', 'DOLARES', 'U_S_M2',
       'AMBIENTES', 'ANTIGUEDAD', 'BAÑOS', 'DIRECCION', 'LOCATION',
       'PUBLICADO', 'PROCESADO', 'URL', 'REVISION', 'NOTA',
       'DIRECCION_NORMALIZADA', 'BARRIO', 'COMUNA', 'CODIGO_POSTAL',
       'CODIGO_POSTAL_ARGENTINO', 'LATITUD', 'LONGITUD', 'PRECIOTEXT_CLEAN',
       'PRECIOTEXTM'],
      dtype='object')

In [10]:
(
    df.groupby("BARRIO").agg(
        PRECO_MEDIO_M2=("PRECIOTEXTM", "mean"),
    )
).head()

,PRECO_MEDIO_M2
BARRIO,
AGRONOMIA,2276.703779
ALMAGRO,3846.216640
BALVANERA,5029.747908
BARRACAS,2817.000357
BELGRANO,9780.558152


## Exercício 5  - agrupar por campos numéricos (decis)

Vamos construir os decis da superfície de dados usando o método `qcut` do numpy.

Isso associa uma categoria a cada registro de dados, onde o valor dessa categoria indica a qual decil esse registro pertence.

Usando o método `groupby`, vamos calcular o preço médio por $m^{2}$ em dólares para cada um dos decis de superfície.

In [34]:
df.columns

Index(['CALLE', 'NUMERO', 'ID_ZONAPRO', 'OPERACION', 'TIPO', 'M2', 'M2CUB',
       'PRECIOTEXT', 'PRECIOARS', 'PRECIOARSM', 'DOLARES', 'U_S_M2',
       'AMBIENTES', 'ANTIGUEDAD', 'BAÑOS', 'DIRECCION', 'LOCATION',
       'PUBLICADO', 'PROCESADO', 'URL', 'REVISION', 'NOTA',
       'DIRECCION_NORMALIZADA', 'BARRIO', 'COMUNA', 'CODIGO_POSTAL',
       'CODIGO_POSTAL_ARGENTINO', 'LATITUD', 'LONGITUD', 'PRECIOTEXT_CLEAN',
       'PRECIOTEXTM'],
      dtype='object')

In [37]:
(df.assign(CUT=pd.qcut(df["M2"], 10)).groupby("CUT").agg(PRECO_MEDIO=("PRECIOTEXTM", "mean")))

C:\Users\levyv\AppData\Local\Temp\ipykernel_15232\4129412142.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("CUT")


,PRECO_MEDIO
CUT,
"(14.999, 35.0]",4656.265213
"(35.0, 39.0]",4295.471399
"(39.0, 43.0]",3503.865741
"(43.0, 48.0]",3033.866684
"(48.0, 54.0]",3291.493085
"(54.0, 64.0]",3991.766550
"(64.0, 74.0]",4159.138966
"(74.0, 88.0]",4021.144715
"(88.0, 123.0]",4610.250705


## Exercício 6  - agrupar por campos do tipo string

Usando o método `groupby`, vamos calcular o preço médio por $m^{2}$ em dólares para cada um dos bairros da cidade.

In [12]:
(
    df.groupby("BARRIO").agg(
        PRECO_MEDIO_M2=("PRECIOTEXTM", "mean"),
    )
).head()

,PRECO_MEDIO_M2
BARRIO,
AGRONOMIA,2276.703779
ALMAGRO,3846.216640
BALVANERA,5029.747908
BARRACAS,2817.000357
BELGRANO,9780.558152


## Exercício 7 - agrupamento por condições personalizadas

Vamos definir uma máscara booleana que é verdadeira para os registros cujo preço em dólares é de até 150.000

Vamos agrupar por este critério e calcular o valor da área mediana para esses dois grupos (preço $\le 150.000$ e preço $ \lt 150.000$).

In [39]:
(df.assign(PRECO_ATE_150=lambda d: d["DOLARES"] < 150000).groupby("PRECO_ATE_150").agg(AREA_MEDIANA=("M2", "median")))

,AREA_MEDIANA
PRECO_ATE_150,
False,86.0
True,43.0
